# Project 6. Distributed Embeddings and Semi-Supervised Learning #

In this assignment, you will use singular value decomposition (SVD) as well as Word2Vec to learn about lexical semantics. You will have to work with "big" data
- big enough that you will have to think carefully about speed and memory. Of particular importance will **sparse** matrix
representations of your data. For this problem you will be submitting pdf version of ipython with outputs along with original ipython. Some particular functions and classes you might need:

- [scipy.sparse.csr_matrix](http://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html) - matrix in compressed sparse row format
- [scipy.sparse.diags](http://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.diags.html) - method for creating sparse diagonal matrices
- [diagonal()](http://docs.scipy.org/doc/numpy/reference/generated/numpy.diagonal.html) - get the diagonal of a matrix
- [sklearn.preprocessing.normalize](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.normalize.html) - efficiently normalize sparse matrices
- [scipy.sparse.csr_matrix.asfptype()](http://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.asfptype.html) - upcast matrix to a floating point format
- [scipy.cluster.vq.kmeans2](http://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.vq.kmeans2.html#scipy.cluster.vq.kmeans2) - Classify a set of observations into k clusters using the k-means algorithm
- [numpy.argsort](http://docs.scipy.org/doc/numpy/reference/generated/numpy.argsort.html) - returns the indices that would sort an array


In [ ]:
%pylab inline
from scipy.sparse.linalg import svds
from scipy.sparse import hstack, diags, csr_matrix
from sklearn.preprocessing import normalize
import numpy as np
import csv
from word2vec import VectorModel
from numpy import *

In [ ]:
def csv2csr(filename):
    word = []
    context = []
    count = []
    with open(filename,'rb') as infile:
        reader = csv.reader(infile)
        for row in reader:
            word.append(int(row[0]))
            context.append(int(row[1]))
            count.append(int(row[2]))
    return csr_matrix((count,(word,context)))

def readVocab(filename):
    vocab = []
    with open(filename,'rb') as vocabfile:
        for line in vocabfile:
            vocab.append(line.split()[0])
    index = dict(zip(range(0,len(vocab)),vocab)) #from numbers to words
    inv_index = {j:i for i,j in index.items()} #from words to numbers
    return index,inv_index


## 1. Loading the Data ##

Call **C=proj4_starter.csv2csr('doc_trips.csv')** to load a sparse matrix $C$ of a word-document counts. The cell $c[i,j]$ should 
hold the count of word $i$ in document $j$.

Call **idx, iidx=proj4_starter.readVocab('vocab.10k')** to load the vocabulary. You get two **dict** objects, mapping between words 
and indices in the matrix $C$. In **C[iidx['Obama'],:]**, you have the document counts for the word *Obama*.

In [ ]:
C = csv2csr('doc_trips.csv')
idx, iidx = readVocab('vocab.10k')
print C.shape

## 2. Cosine Similarity ##

The *cosine similarity* of two vectors $u$ and $v$ is defined as 
$$\frac{\sum_{i}u_iv_i}{\sqrt{\sum_iu_i^2\sum_iv_i^2}}$$

** Deliverable 2a** Consider the words *coffee, play, crazy, facebook*, and *hermana* (Spanish for *sister*). For each of them, find the 10 most similar words according to cosine similarity of the rows in $C$. (2 points)

**Hint** The size of the vocabulary is nearly 10,000 words. You do not want to compute and store the entire $10K\times 10K$ matrix 
of cosine similarities. Rather, you want to compute them on demand for a given row of the matrix. You may also want to do some
precomputation to take care of denominator in advance. Whatever you do, don't lose the sparsity of $C$, or you will not be able 
to store it.

**Sanity check** For *facebook*, the top 5 words I get are *facebook page on twitter deleted instagram*

In [ ]:
# Here is the word list
word_list = ['coffee','play','crazy','facebook','hermana']

In [ ]:
def normalizeRow(x):
    '''
    Normalize each row of x
    '''
    return diags(np.array(1./(1e-6+np.sqrt(x.multiply(x).sum(axis=1))))[:,0],0) * x

def computeCosSimPerWord(word_idx, x):
    '''
    For a given data matrix, compute cosine similarity between the word with index "word_index" and all words 
    (including itsself)
    
    Should return a 1-D np.array, not a matrix.
    '''
    
    pass

normalizedC = normalizeRow(C)

printSimilarWords  is used to print the top 10 similar words to a given word

In [ ]:
def printSimilarWords(x, word_list, sim_func, vocab=idx, ivocab=iidx):
    for word in word_list:
        print word, ':', 
        word_idx = ivocab[word]
        sim_idx = np.argsort(-sim_func(word_idx, x))[:10]
        for word2_idx in sim_idx:
            print vocab[word2_idx],
        print ''

In [ ]:
printSimilarWords(normalizedC, word_list, sim_func=computeCosSimPerWord)

** Deliverable 2b ** Come up with five words of your own that you think might be interesting, and list the top 10 most similar for each. Try to choose a few different types of words, such as verbs, adjectives, names, emotions, abbreviations, or alternative spellings. (1 point)

In [ ]:
additional_word_list = []

In [ ]:
printSimilarWords(normalizedC, additional_word_list, sim_func=computeCosSimPerWord)

## 3. Document Co-occurence ##

Compute the document co-occurence matrix $D$, where $d_{i,j}$ is the probability $P(w_j|w_i)$ that word $j$ appears in a tweet, 
given that word $i$ appears. To do this, first compute the co-occurence counts $CC^\top$. Substract the diagonal, then normalize 
each row. 

Note: it is possible to smooth this probability, but if you naively add some number to the matrix, you will lose sparsity 
and memory will blow up. You can do it unsmoothed. However, smoothing is not required here.

** Deliverable 3** For each of the 10 examples above (my five words and your five words), find the 10 most similar words according to cosine similarity of the rows of $D$. (2 points)

**Sanity check** For *facebook*, the 5 words I get are *facebook instagram twitter tv youtube*

In [ ]:
def computeCooccurMatrix(C):
    '''
    Compute the co-occurence matrix D
    '''
    D = None
    return D

D = computeCooccurMatrix(C)
normalizedD = normalizeRow(D)

In [ ]:
word_list = word_list + additional_word_list

In [ ]:
printSimilarWords(normalizedD, word_list, sim_func=computeCosSimPerWord)

## 4. Latent Semantic Analysis ##

Perform truncated SVD (**scipy.sparse.linalg.svds**) to obtain $USV^\top\approx C$ using $K=10$. Each row vector $u_i$
is a description of the word $i$. You can compute similarity between pairs of words using the squared Euclidean norm 
$\|u_i-u_j\|^2_2$.

** Deliverable 4(a)** For each of the 10 examples above, find the 10 most similar words according to squared Euclidean distance in $U$. (3 points)

**Sanity check** For *facebook*, the top 5 words are *facebook ex harry calls snap *

In [ ]:
def computeEuclidDist(word_idx, U,with_S = False):
    '''
    Compute the Euclid distance bwteen word with index "word_idx" and all words 
    (including itself)
    
    Args:
      word_idx - the word index
      U - latent representation of words
    Return:
        Euclidean distance from representation of word_idx to all words
    '''
    dis = 0
    return dis

In [ ]:
'''
Once you finish the function computeEuclidDist, run the following code directly to print results
'''
Cfp= C.asfptype()
U = svds(Cfp, 10)

In [ ]:
printSimilarWords(U, word_list, sim_func=lambda word_idx, U : -computeEuclidDist(word_idx,U))

** Deliverable 4(b) ** Now compute the same SVD with $K=50$, and again find the 10 most similar words according to Euclidean distance $U$. (1 point)

In [ ]:
# your code here

** Deliverable 4(c) ** Now compute the SVD of the matrix $\mathbf{D}$, using with $K = 10$, and $K = 50$. Report 
the most similar words to each of the example words according to Euclidean distance in $U$. (1 point)

In [ ]:
# your code here for K = 10

In [ ]:
# your code here for K = 50

In [ ]:
# optionally, try K = 100 and see if it's even better

## 5. Local Context ##

Local context captures the frequency with which words appear in each others’ immediate context. We have provided a CSV file (succ_trips_50k.csv)  in which each line contains a triple $\langle x,y,z\rangle$, 
where $x$ and $y$ are term IDs and $z$ is the count of times where $y$ immediately follows $x$ . 
The vocabulary has now increased to 50K words. There is an associated vocabulary file, **vocab.50k**.

**Deliverable 5a **
Build a sparse matrix $\mathbf{E}$ from these triples. Normalize the rows of $\mathbf{E}$, such that $e_{i,j}=\frac{n(i,j)}{n(i)}$, the probability of seeing word $j$ given that you have just seen word $i$. 
Now form a matrix $\mathbf{F} = [\mathbf{E}~ \mathbf{E}’]$ by horizontally concatenating the normalized matrix $\mathbf{E}$. You will perform sparse singular value decomposition on $\mathbf{F}$. (2 points)

**Hint** make sure you are using a sparsity-preserving operation to combine E and E'!

In [ ]:
idx_50, iidx_50 = readVocab('vocab.50k')
E = csv2csr('succ_trips_50k.csv')

In [ ]:
def constructF(E):
    '''
    Finish the following code to construct F from E
    '''
   
    Enorm =None
    return Enorm
    
F = constructF(E)

** Deliverable 5b ** For $K = 10$ and $K = 50$ compute the top 10 synonyms for each of your ten words. (1 point)

In [ ]:
# K = 10
U_f,S_f,V_f = svds(F, 10)
# note that we have to insert the new vocabulary as an optional argument
printSimilarWords(U_f, word_list, vocab=idx_50, sim_func=lambda word_idx, U : -computeEuclidDist(word_idx,U))

In [ ]:
# your code for K = 50 here

**Deliverable 5c ** Overall, which set of synonyms looks best to you? 
Count how many of the top 5 synonyms for *coffee* and   *crazy*
have the same majority part of speech (e.g., *play* is a verb) as the cue word.
Use the tagset from the [Twitter POS paper](http://www.cc.gatech.edu/~jeisenst/papers/acl2012pos.pdf). Does local context or document context do better at matching the POS of the cue words? Why? (Consider first context provided by wordnet as majority POS). (3 points)

*(Your answer here)*

## 6. Word2Vec ##
As mentioned in the class Word2Vec are distributed continuous vector representations  for words.
In this part, you will be building and training Word2Vec module keeping in mind various hyper parameters such as min_count, window_size etc.(10 points)
You will be using Gensim http://radimrehurek.com/gensim/ to train your model.

- For instructions on how to install gensim, see [here](https://radimrehurek.com/gensim/install.html). 
- I strongly recommend that you test your install by downloading the [source](http://pypi.python.org/pypi/gensim) and running ```python setup.py test```
- For a tutorial on how to use gensim for word embeddings, see [here](http://rare-technologies.com/word2vec-tutorial/)
- It is recommended that you have a working c compiler, so that the much faster cythonized gensim can run. This is transparent to you, but does require that you have a c compiler installed.


In [1]:
from nltk.tokenize import word_tokenize

In [ ]:
def read_data(filename):
        print "Opening the file..."

        X_train = []

        f = open(filename,'r')
        count = 0
        for line in f.readlines():
            sentence = []
            line = line.strip()
            if not line: continue
            try:
                sentence = word_tokenize(line)
            except:
                pass
            if(len(sentence) > 2):
                count =count+1
                X_train.append(array(sentence))
            # else:
            #      print "No words"
            #      print sentence

        print "File successfully read"
        print count , "sentences"
        f.close()
        return array(X_train)

In [ ]:
sentences = read_data('wiki.train.en')

To build and train model you will be using [Gensim](http://radimrehurek.com/gensim/). 

This [tutorial](https://radimrehurek.com/gensim/models/word2vec.html) may help.

**Deliverable 6a**: Build a model with embedding size = 10, and build a vocabulary from the data, with min_count = 10. Print the vocabulary size, which should be between 9000 and 10000. (1 point)

In [2]:
# your code here

**Deliverable 6b**: Train your model for one iteration, and report the similarity between the words ("cat" and "dog"), ("night" and "day"), and ("can" and "fish"). (2 points)

**Hint**: before training on the whole dataset, train on sentences[0:5] to make sure it works and isn't too slow. On my laptop, one iteration takes 2-3 seconds. 

**Sanity check** the similarity I get between "cat" and "dog" is 0.13

In [3]:
# your code here

**Deliverable 6c** Modify the code below to compare the performance of different embedding sizes. (2 points)

In [ ]:
def trainMod(model,sentences,max_its=10):
    # your code to build the vocabulary
    scores = []
    times = []
    for _ in xrange(max_its):
        # your code to train the model
        scores.append(sum(model.score(sentences)))
        times.append(model.total_train_time)
    return scores,times

In [ ]:
all_k = [20,40,100] # embedding sizes
models = []
for k in all_k:
    model = #your code to create a model here
    models.append(model)
    scores,times = trainMod(model,sentences)
    plt.plot(times,scores,'o-');
    model.save('model-%d'%(k))
    print 'done with',k
plt.legend(['k=%d'%(k) for k in all_k],loc='lower right');
plt.xlabel('time')
plt.ylabel('log likelihood')

**Deliverable 6d** Use the ```most_similar``` function to find similar words to the items in your word_list which are in the vocabulary for your word2vec models. Compare these most similar words with the outputs of the methods tried earlier in the assignment. You may train the word2vec models for longer if you wish, or change any of the parameters. (2 points)

In [ ]:
# your code here

(your explanation here)

One of the claims about word2vec word embeddings is that they can solve analogy problems, like "man:woman::king:queen"

[This tutorial](http://rare-technologies.com/word2vec-tutorial/) contains example code showing how to use word vectors to try to solve analogies.

**Deliverable 6e** Using the three models trained in the previous deliverable, see if each can solve the analogies man:woman::king:queen and architect:building::painter:painting. Invent two more analogies, and see which of the three models can solve them. You can Google analogies, but you may have to search for a while to find analogies where all four elements are in the vocabulary. You can also retrain the models with a larger vocabulary if you want. (2 points)

In [5]:
# your code here

# 7. Semi-supervised learning #

Now you will use the word embeddings to try to improve your dependency parser from problem set 5.

You will do this by clustering words according to their embedding.

The code below run the clustering algorithm.

In [7]:
from scipy.cluster.vq import kmeans2

In [ ]:
# choose a model to use here
word_vectors = model.syn0
# run the clustering algorithm
centroids, labels = kmeans2(word_vectors,num_clusters,iter=100,minit='points')

**Deliverable 7a** Modify the code below to show the words in the same cluster as a query word. Find the words in the same cluster as "coffee", "computer", and "red" for at least one of your models, plus any other words of interest. (3 points)

In [9]:
def getWordsInSameCluster(word,model,labels):
    # you will use model.index2word to make this function
    # return a list of words in the same cluster as word
    pass

## Improving dependency parsing ##

Now incorporate these cluster features into your best performing parser from project 5, based on the dev data. Add a feature for each cluster/tag pair, e.g., C175/N, C189/V, etc. You will then compute the accuracy with training sets of various sizes, comparing the performance of your model with and without the cluster features.

**Deliverable 7b ** Build training sets including the first 50, 100, 200, 500, and 1000 *sentences* (not words). 
Train your tagger on each training set, using your original features, and plot the accuracy on the development set. 
Then retrain you tagger, including the new word cluster features, and plot accuracy on the development set on the same plot. 
Run for at least 10 iterations in each case.

You may want to try larger numbers of clusters to improve performance. 

You can even include the results from multiple clusterings with different numbers of clusters (50, 100, 200, 500, ...), 
using features like C43-50/N (cluster 43 of 50, with tag N), C377-500/V (cluster 377 of 500, with tag V).

You can also use clusters for the features of neighboring words.  (6 points)

In [10]:
# your code here

In [ ]:
print 'Train 100'
dp = depp.DependencyParser(feature_function=BakeOffFeats())
dp.read_data("english",100)
tr_acc ,dv_acc  = dp.train_perceptron(10) 

print 'Train 100'
dp = depp.DependencyParser(feature_function=W2V_ClusterFeats())
dp.read_data("english",100)
tr_acc ,dv_acc  = dp.train_perceptron(10) 

## 8. Get creative ##

This part is mandatory for 7650 and optional for 4650.

**Deliverable 8 ** K-means clustering is one way to find similarity. This part is opened-ended and requires you to come up with creative solutions of using distributed vectors to improve the results. In addition to implementing your idea, provide explanation of what you did, and why you thought it should work.

The top 3 selected ideas (based mostly on performance, but also on creativity) will get an extra point towards their final score.(5 points)